In [6]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600  ~/.kaggle/kaggle.json

In [7]:
# Импортируем датасет с kaggle
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('tongpython/cat-and-dog', 'data')

In [8]:
# Разархивируем данные
import zipfile
path_to_zip_file = '/content/data/cat-and-dog.zip'
with zipfile.ZipFile(path_to_zip_file, 'r') as z:
  z.extractall('data1')

In [15]:
# Удаляем ненужные файлы
!rm /content/data1/test_set/test_set/cats/_DS_Store
!rm /content/data1/test_set/test_set/dogs/_DS_Store
!rm /content/data1/training_set/training_set/cats/_DS_Store
!rm /content/data1/training_set/training_set/dogs/_DS_Store

rm: cannot remove '/content/data1/test_set/test_set/cats/_DS_Store': No such file or directory
rm: cannot remove '/content/data1/test_set/test_set/dogs/_DS_Store': No such file or directory
rm: cannot remove '/content/data1/training_set/training_set/cats/_DS_Store': No such file or directory
rm: cannot remove '/content/data1/training_set/training_set/dogs/_DS_Store': No such file or directory


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision as tv

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

from torch.cuda.amp import autocast, GradScaler

In [17]:
class Dataset_(torch.utils.data.Dataset):
    def __init__(self, path_dir1:str, path_dir2:str):
        super().__init__()

        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2

        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))

    def __len__(self):  #Получаем длину датасета
        return len(self.dir1_list) + len(self.dir2_list)

    def __getitem__(self, idx):   #Получаем элемент датасета

        if idx < len(self.dir1_list): # Определяем метку класса, после находим путь к изображению
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        else:
            class_id = 1
            idx -= len(self.dir1_list)
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])

        # загружаем и нормализуем изображение
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))

        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)

        return {'img': t_img, 'label': t_class_id}


In [18]:
train_dogs_path = '/content/data1/training_set/training_set/dogs'
train_cats_path = '/content/data1/training_set/training_set/cats'
test_dogs_path = '/content/data1/test_set/test_set/dogs'
test_cats_path = '/content/data1/test_set/test_set/cats'
# Создаем датасеты
train_ds_catsdogs = Dataset_(train_dogs_path, train_cats_path)
test_ds_catsdogs = Dataset_(test_dogs_path, test_cats_path)

In [19]:
len(train_ds_catsdogs)

8005

In [20]:
len(test_ds_catsdogs)

2023

In [21]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    train_ds_catsdogs, shuffle=True,
    batch_size=batch_size, num_workers=1, drop_last=True
)
test_loader = torch.utils.data.DataLoader(
    train_ds_catsdogs, shuffle=True,
    batch_size=batch_size, num_workers=1, drop_last=False
)

In [22]:
# Прописываем архитектуру НС
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.act = nn.LeakyReLU(0.2)
        self.maxpool = nn.MaxPool2d(2,2)
        self.conv0 = nn.Conv2d(3, 128, 3, stride=1, padding=0)
        self.conv1 = nn.Conv2d(128, 128, 3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(128, 128, 3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(128, 256, 3, stride=1, padding=0)

        self.adaptivepool = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(256, 20)
        self.linear2 = nn.Linear(20, 2)

    def forward(self, x):


        out = self.conv0(x)
        out = self.act(out)
        out = self.maxpool(out)

        out = self.conv1(out)
        out = self.act(out)
        out = self.maxpool(out)

        out = self.conv2(out)
        out = self.act(out)
        out = self.maxpool(out)

        out = self.conv3(out)
        out = self.act(out)

        out = self.adaptivepool(out)
        out = self.flatten(out)
        out = self.linear1(out)
        out = self.act(out)
        out = self.linear2(out)

        return out

In [23]:
# Подсчет параметров НС
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [24]:
model = ConvNet()

In [25]:
model

ConvNet(
  (act): LeakyReLU(negative_slope=0.2)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (adaptivepool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=256, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=2, bias=True)
)

In [26]:
count_parameters(model)

599102

In [27]:
for sample in train_loader:
    img = sample['img']
    label = sample['label']
    model(img)
    break

In [28]:
# Функция потерь и оптимизатор
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [29]:
# Метрика точности классификации
def accuracy(pred, label):
    answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
    return answer.mean()

In [30]:
device = 'cuda' # if torch.cuda.is_available() else 'cpu'
model = model.to(device)
loss_fn = loss_fn.to(device)

In [31]:
use_amp = True
scaler = torch.cuda.amp.GradScaler()

In [32]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

In [35]:
epochs = 15
# Цикл обучения НС
for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        img, label = sample['img'], sample['label']
        label = F.one_hot(label, 2).float()
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        with autocast(use_amp):
            pred = model(img)
            loss = loss_fn(pred, label)

        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item

        scaler.step(optimizer)
        scaler.update()

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
    print(loss_val/len(train_loader))
    print(acc_val/len(train_loader))

  0%|          | 0/500 [00:00<?, ?it/s]

<ipython-input-29-d3b8e739903a>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)


0.46814370560646057
0.781875


  0%|          | 0/500 [00:00<?, ?it/s]

0.4385087276697159
0.79625


...

0.18174649873375892
0.924375


  0%|          | 0/500 [00:00<?, ?it/s]

0.15182006457448005
0.938


  0%|          | 0/500 [00:00<?, ?it/s]

0.1470711371153593
0.93975


In [36]:
loss_val = 0
acc_val = 0
# Цикл валидации НС
for sample in (pbar := tqdm(test_loader)):
    with torch.no_grad():
        img, label = sample['img'], sample['label']

        label = F.one_hot(label, 2).float()
        img = img.to(device)
        label = label.to(device)
        pred = model(img)

        loss = loss_fn(pred, label)
        loss_item = loss.item()
        loss_val += loss_item

        acc_current = accuracy(pred.cpu(), label.cpu())
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
print(loss_val/len(train_loader))
print(acc_val/len(train_loader))

  0%|          | 0/501 [00:00<?, ?it/s]

<ipython-input-29-d3b8e739903a>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)


0.10385676965070888
0.961
